In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/RBE595AI_Final/'

Mounted at /content/drive
/content/drive/MyDrive/RBE595AI_Final


In [3]:
import numpy as np
import os

image_files = os.listdir('data_road/training/image_2')
labels = []
for file in image_files:
  parts = file.split('_')
  base = parts[1].split('.')
  name = f'{parts[0]}_road_{base[0]}.npy'
  labels.append(name)

image_files = np.array([f'data_road/training/image_2/{x}' for x in image_files])
labels = np.array([np.load(f'data_road/training/labels/{x}') for x in labels])

In [4]:
import tensorflow as tf

data = tf.data.Dataset.from_tensor_slices((image_files, labels))

@tf.function
def readimage(image_file, label):
    image = tf.image.resize_with_pad(tf.io.decode_jpeg(tf.io.read_file(image_file)), 384, 1248) / 255
    return image, label

batch_size = 1
shuffle_buffer_size = 289
train_ds_size = int(289 * 0.75)
val_ds_size = 289 - train_ds_size

shuffled_ds = data.shuffle(shuffle_buffer_size).map(readimage, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = shuffled_ds.take(train_ds_size).batch(batch_size)
val_ds = shuffled_ds.skip(train_ds_size).take(val_ds_size).batch(batch_size)

In [5]:
from tensorflow.python.ops.gen_array_ops import strided_slice
from tensorflow.keras.layers import Dense, Conv2D, LSTM, MaxPool2D, Flatten, InputLayer, Conv2DTranspose, RandomRotation, Dropout
from tensorflow.keras.models import Sequential


def create_model():
    model = Sequential([
        InputLayer(input_shape=(384,1248,3)),
        RandomRotation(0.2),
        Conv2D(3, (4,4), strides=2, padding='same', activation='elu'),
        Conv2D(3, (3,3), padding='same', activation='elu'),
        Conv2D(6, (4,4), strides=2, padding='same', activation='elu'),
        Conv2D(6, (3,3), padding='same', activation='elu'),
        Conv2D(12, (4,4), strides=2, padding='same', activation='elu'),

        Conv2D(128, (3,3), dilation_rate=(1,1), padding='same', activation='elu'),
        Dropout(0.25),
        Conv2D(128, (3,3), dilation_rate=(1,1), padding='same', activation='elu'),
        Dropout(0.25),
        Conv2D(128, (3,3), dilation_rate=(1,2), padding='same', activation='elu'),
        Dropout(0.25),
        Conv2D(128, (3,3), dilation_rate=(2,4), padding='same', activation='elu'),
        Dropout(0.25),
        Conv2D(128, (3,3), dilation_rate=(4,8), padding='same', activation='elu'),
        Dropout(0.25),
        Conv2D(128, (3,3), dilation_rate=(8,16), padding='same', activation='elu'),
        Dropout(0.25),
        Conv2D(128, (3,3), dilation_rate=(16,32), padding='same', activation='elu'),
        Dropout(0.25),
        Conv2D(128, (3,3), dilation_rate=(1,1), padding='same', activation='elu'),
        Dropout(0.25),
        Conv2D(128, (1,1), dilation_rate=(1,1), padding='same', activation='elu'),

        Conv2DTranspose(6, (4,4), strides=2, padding='same', activation='elu'),
        Conv2DTranspose(6, (3,3), padding='same', activation='elu'),
        Conv2DTranspose(3, (4,4), strides=2, padding='same', activation='elu'),
        Conv2DTranspose(3, (3,3), padding='same', activation='elu'),
        Conv2DTranspose(8, (4,4), strides=2, padding='same', activation='elu'),
        Conv2DTranspose(2, (3,3), padding='same', activation='softmax')
    ])

    decay = tf.keras.optimizers.schedules.PolynomialDecay(0.0005, 289, end_learning_rate=0, power=0.9)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=decay, epsilon=0.1), 
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_rotation (RandomRota  (None, 384, 1248, 3)     0         
 tion)                                                           
                                                                 
 conv2d (Conv2D)             (None, 192, 624, 3)       147       
                                                                 
 conv2d_1 (Conv2D)           (None, 192, 624, 3)       84        
                                                                 
 conv2d_2 (Conv2D)           (None, 96, 312, 6)        294       
                                                                 
 conv2d_3 (Conv2D)           (None, 96, 312, 6)        330       
                                                                 
 conv2d_4 (Conv2D)           (None, 48, 156, 12)       1164      
                                                        

In [ ]:
tf.keras.backend.clear_session()

callbacks = [
    tf.keras.callbacks.TensorBoard(),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join('ckpt-FCN','training-FCN'), 
        save_weights_only=True)
]

# model.load_weights(os.path.join('ckpt', 'training-3'))
model.fit(x=train_ds, batch_size=batch_size, epochs=100000, callbacks=callbacks, validation_data=val_ds, validation_freq=1000, verbose=1)

Streaming output truncated to the last 5000 lines.
Epoch 412/100000
216/216 [==============================] - 8s 35ms/step - loss: 0.1708 - accuracy: 0.9222
Epoch 413/100000
216/216 [==============================] - 8s 36ms/step - loss: 0.1736 - accuracy: 0.9218
Epoch 414/100000
216/216 [==============================] - 8s 35ms/step - loss: 0.1709 - accuracy: 0.9233
Epoch 415/100000
216/216 [==============================] - 8s 36ms/step - loss: 0.1724 - accuracy: 0.9224
Epoch 416/100000
216/216 [==============================] - 7s 35ms/step - loss: 0.1720 - accuracy: 0.9228
Epoch 417/100000
216/216 [==============================] - 8s 35ms/step - loss: 0.1711 - accuracy: 0.9238
Epoch 418/100000
216/216 [==============================] - 8s 36ms/step - loss: 0.1719 - accuracy: 0.9231
Epoch 419/100000
216/216 [==============================] - 8s 35ms/step - loss: 0.1741 - accuracy: 0.9225
Epoch 420/100000
216/216 [==============================] - 8s 35ms/step - loss: 0.1742 - acc

In [10]:

# model.load_weights(os.path.join('ckpt-FCN', 'training-FCN'))
val_np = val_ds.as_numpy_iterator()
image, label = list(val_np)[0]
# predictions = model.predict(val_ds)

In [14]:
prediction = model.predict(image, batch_size=1)

In [ ]:
def to_image(prediction):
  result = np.array()
  for i in range(prediction):
    for j in range(prediction):
